ERA5

- mean, median, std, min, max for precipitation values by year and year-month

- maps for both results


### Yearly

### Monthly

In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\UV_wind\2wind_speed_direction"
output_folder = r"D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year and month as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in tqdm(np.unique(ds['month'].values), desc=f"Year {year_val}", leave=False):
        ds_month = ds.sel(valid_time=(ds['month'] == month_val))
        if ds_month['valid_time'].size == 0:
            continue

        wind_data = ds_month['wind_speed'].values

        # Compute daily statistics
        mean = np.nanmean(wind_data, axis=0)
        median = np.nanmedian(wind_data, axis=0)
        std = np.nanstd(wind_data, axis=0)
        max_ = np.nanmax(wind_data, axis=0)
        min_ = np.nanmin(wind_data, axis=0)

        # Create Dataset for this month's stats
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame
        stats_df = stats.to_dataframe().reset_index()

        # Add time labels
        stats_df['Year'] = year_val
        stats_df['Month'] = int(month_val)

        # Set multi-index
        stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"monthly_wind_speed_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:39<13:59, 39.99s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly\monthly_wind_speed_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly\monthly_wind_speed_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [07:06<00:20, 20.59s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly\monthly_wind_speed_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [07:27<00:00, 20.36s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly\monthly_wind_speed_stats_2024.nc

🎉 All yearly files saved successfully!


### Daily

In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\UV_wind\2wind_speed_direction"
output_folder = r"D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year, month, day as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month,
        day=ds['valid_time'].dt.day
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in np.unique(ds['month'].values):
        days_in_month = np.unique(ds['day'].values[ds['month'].values == month_val])
        
        # Loop over each day in the month with a progress bar
        for day_val in tqdm(days_in_month, desc=f"Month {month_val}", leave=False):
            ds_day = ds.sel(
                valid_time=(ds['month'] == month_val) & (ds['day'] == day_val)
            )
            if ds_day['valid_time'].size == 0:
                continue
            
            wind_data = ds_day['wind_speed'].values

            # Compute daily statistics
            mean = np.nanmean(wind_data, axis=0)
            median = np.nanmedian(wind_data, axis=0)
            std = np.nanstd(wind_data, axis=0)
            max_ = np.nanmax(wind_data, axis=0)
            min_ = np.nanmin(wind_data, axis=0)

            # Create Dataset for this day's stats
            stats = xr.Dataset({
                'Mean': (['latitude', 'longitude'], mean),
                'Median': (['latitude', 'longitude'], median),
                'Std': (['latitude', 'longitude'], std),
                'Max': (['latitude', 'longitude'], max_),
                'Min': (['latitude', 'longitude'], min_)
            }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

            # Convert to DataFrame
            stats_df = stats.to_dataframe().reset_index()

            # Add time labels
            stats_df['Year'] = year_val
            stats_df['Month'] = month_val
            stats_df['Day'] = int(day_val)

            # Set multi-index
            stats_df = stats_df.set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
            df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"daily_wind_speed_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:35<12:34, 35.94s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly\daily_wind_speed_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly\daily_wind_speed_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [11:45<00:32, 32.81s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly\daily_wind_speed_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [12:20<00:00, 33.67s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly\daily_wind_speed_stats_2024.nc

🎉 All yearly files saved successfully!
